<a href="https://colab.research.google.com/github/AbhijeetBeedikar/Simple-Racetrack-/blob/main/Simple_Racetrack_RL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import random
from collections import defaultdict


def start_and_end(track):
  start = ()
  end = ()
  for i in range(len(track)):
    for j in range(len(track[i])):
      if list(track[i])[j] == "S":
        start = (j,i)
      if list(track[i])[j] == "F":
        end = (j,i)
  return start,end


def step(state,policy):
  if policy == "b":
    action = np.random.choice(list(actions.keys()))
  elif policy == "pi":
    action = pi[state]
  nvx = min(5,max(-5,state[2]+actions[action][0]))
  nvy = min(5,max(-5,state[3]+actions[action][1]))
  nx = state[0] + nvx
  ny = state[1] + nvy

  reward = -1 # reward for completing the task is set to be 3 because it takes multiple steps to complete the task,
              # so the reward must be higher than the negative inverse of the punishment per step to encourage the agent
              # to have a greater bias towards actions that lead to completing the task
  termination = False
  if nx < 0 or ny < 0 or nx >= len(track[0]) or ny >= len(track): #out of bounds case
    termination = True

  elif list(track[ny])[nx] == "#": #out of track case
    termination = True

  elif (fx-state[0] == 0 and nx-state[0] == 0 # old state, new state and final state are along a vertical line
        and abs(ny-state[1]) >= abs(fy-state[1]) # distance from old state to new state must be >= distance from old state to final
        and ((ny-state[1] >=0 and fy-state[1] >=0 ) or (ny-state[1] <=0 and fy-state[1] <=0))): # the car must move along the direction of the final state
      reward = 3
      termination = True

  elif (fy-state[1] == 0 and ny-state[1] == 0 # old state, new state and final state are along a horizontal line
        and abs(nx-state[0]) >= abs(fx-state[0]) # distance from old state to new state must be >= distance from old state to final
        and ((nx-state[0] >=0 and fx-state[0] >=0 ) or (nx-state[0] <=0 and fx-state[0] <=0))): # the car must move along the direction of the final state
      reward = 3
      termination = True

  elif (fx-state[0] != 0 and nx-state[0] != 0
        and (fy-state[1])/(fx-state[0]) == (ny-state[1])/(nx-state[0])  #ensuring that the old state, new state and inal state lie along the same diagonal
        and (((ny-state[1])**2+(nx-state[0])**2) >= ((fy-state[1])**2+(fx-state[0])**2)) #making sure the distance from old state to new state is larger than the distance from old state to finish
        and ((fy-state[1] >= 0 and ny-state[1]>=0) or (fy-state[1] <= 0 and ny-state[1]<=0))): #making sure that the car moves in the same direction as the finish line and not in the opposite direction
    reward = 3
    termination = True
  return (nx,ny,nvx,nvy), action, reward, termination


def off_policy_MCC(episodes, gamma):
  pi = defaultdict(lambda:5) # action arbitrarily set to (0,0) by default
  Q = defaultdict(lambda:0.0)
  C = defaultdict(lambda:0.0)
  for i in range(episodes):
    W = 1
    G = 0
    # generating the episode according to behavior policy till termination
    terminate = False
    state = (x,y,vx,vy)
    episode = []
    while terminate == False:
      prev_state = state
      state, action, reward, terminate = step(state,"b")
      episode.append((prev_state,action,reward))

    # Policy control
    for j in reversed(range(len(episode))):
      s,a,r = episode[j]

      # updates
      G = r + gamma*G
      C[s,a] += W
      Q[s,a] += (W/C[s,a])*(G-Q[s,a])


      # greedy selection
      maxQ = []
      for k in actions.keys():
        maxQ.append(Q[s,k])
      if Q[s,a] == max(maxQ):
        pi[s] = a # better to explore further into the episode so we make the greedy equivalent action taken by pi
      else:
        tie = [i for i in range(len(maxQ)) if maxQ[i] == max(maxQ)]
        pi[s] = np.random.choice(tie) + 1 # prevents arbitrary bias towards a particular greedy equivalent action
      if a != pi[s]:
        break

      W *= 9 # Since pi is a greedy deterministic policy, pi(a|s) = 0 for all actions except the greedy action
            # Thus if any action that has pi(a|s) = 0 is taken, then the loop is automatically broken (no updates occur since W would become 0)
            # So, pi(a|s) = 1 when updating W for an episode. b(a|s) = 1/9 since behavior policy chooses randomly from 9 actions
            # THerefore we get W *= 1/(1/9) = 9 --> W*=9

  return Q, pi

# setting up the environment
actions = {1:(-1,1), 2: (0,1), 3: (1,1), 4: (-1,0), 5: (0,0), 6: (1,0), 7: (-1,-1), 8: (0,-1), 9: (1,-1)}
track = [
        "######F",
        "#.....#",
        "#.....#",
        "#..S..#",
        "#######"
    ]
vx = 0
vy = 0
x,y = start_and_end(track)[0]
fx,fy = start_and_end(track)[1]

# Off-policy Every Visit MCC
Q, pi = off_policy_MCC(10000,1)

print(len(Q), dict(Q))
print(len(pi), dict(pi))

# evaluating the mean return of the agent when following policy pi
fin_r = 0
ret = []
for i in range(1000):
  # generating the episode according to behavior policy till termination
  terminate = False
  state = (x,y,vx,vy)
  episode = []
  cum_r = 0
  force = 0
  while terminate == False or force == 100:
    prev_state = state
    state, action, reward, terminate = step(state,"pi")
    episode.append((prev_state,action,reward))
    cum_r += reward
    force+=1
  ret.append(cum_r)
print(np.mean(ret))
print(episode)


963 {((4, 2, 1, -1), np.int64(7)): -1.0, ((4, 2, 1, -1), 1): 0.0, ((4, 2, 1, -1), 2): 1.0, ((4, 2, 1, -1), 3): -1.0, ((4, 2, 1, -1), 4): 1.0, ((4, 2, 1, -1), 5): 2.0, ((4, 2, 1, -1), 6): -1.0, ((4, 2, 1, -1), 8): -1.0, ((4, 2, 1, -1), 9): 3.0, ((2, 3, 0, 0), np.int64(1)): -1.0, ((2, 3, 0, 0), 2): -1.0, ((2, 3, 0, 0), 3): -1.0, ((2, 3, 0, 0), 4): 0.0, ((2, 3, 0, 0), 5): 0.0, ((2, 3, 0, 0), 6): 0.0, ((2, 3, 0, 0), 7): 0.0, ((2, 3, 0, 0), 8): 0.0, ((2, 3, 0, 0), 9): 0.1, ((4, 2, 1, 0), np.int64(9)): -1.0, ((4, 2, 1, 0), 1): 0.0, ((4, 2, 1, 0), 2): 0.0, ((4, 2, 1, 0), 3): -1.0, ((4, 2, 1, 0), 4): 0.0, ((4, 2, 1, 0), 5): 0.0, ((4, 2, 1, 0), 6): -1.0, ((4, 2, 1, 0), 7): 0.0, ((4, 2, 1, 0), 8): 2.0, ((3, 3, 0, 0), np.int64(2)): -1.0, ((3, 3, 0, 0), 1): -1.0, ((3, 3, 0, 0), 3): -1.0, ((3, 3, 0, 0), 4): 0.0, ((3, 3, 0, 0), 5): 1.0, ((3, 3, 0, 0), 6): 0.6249999999999999, ((3, 3, 0, 0), 7): -1.0, ((3, 3, 0, 0), 8): 0.3571428571428572, ((3, 3, 0, 0), 9): 2.0, ((2, 1, -1, -1), np.int64(4)): -1.0, (